In [ ]:
import os
import importlib

import pandas as pd
import numpy as np
import geopy
from geopy import distance
from nile.api.v1 import filters as nf
from nile.api.v1 import extractors as ne
from nile.api.v1 import aggregators as na
from qb2.api.v1 import filters as qf

from projects.common import geo
from projects.common.nile import dates, test_utils
from projects.cpo_model import project_config, default_values, common

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
_ = importlib.reload(default_values)
_ = importlib.reload(project_config)
_ = importlib.reload(common)
_ = importlib.reload(test_utils)
_ = importlib.reload(geo)

#### Declare non-constant objects

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    pool='taxi-delivery',
    extra_requirements=['numpy'],
)

#### Declare constants

In [ ]:
DATE_START = '2020-11-02'
DATE_END = '2020-11-15'
DTTM_START = '2020-11-02T00:00:00Z'
DTTM_END = '2020-11-15T00:00:00Z'

REGION_MAP = {
    213: 'moscow', 1: 'moscow_and_mo',
    2: 'saint-petersburg', 10174: 'saint-petersburg_and_lo'
}

### Get data from YT

In [ ]:
job = cluster.job()

In [ ]:
# Source tables.
market_deliveries = job.table(
    default_values.YT.MARKET_DELIVERIES
)

In [ ]:
# Filter out Moscow.
a = (
    market_deliveries
    .filter(
        nf.custom(lambda x: DATE_START <= test_utils.to_string(x)[:10] <= DATE_END
                  if x else False, 'creation_local_datetime')
    )
    .project(
        ne.all(),
        region=ne.custom(lambda x: REGION_MAP.get(x, 'other'), 'delivery_region_id')
    )
    .put(os.path.join(default_values.YT.PROJECT_FOLDER,
                      'orders_{}_{}'.format(DATE_START, DATE_END)))
)
(
    a
    .groupby('region', 'fulfillment_name')
    .aggregate(cnt=na.count())
    .put(os.path.join(default_values.YT.PROJECT_FOLDER,
                      'fullfillment_names_by_region_{}_{}'.format(DATE_START, DATE_END)))
)

In [ ]:
job.flow_graph

In [ ]:
job.run()

### Get data

In [ ]:
df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/orders_2020-11-02_2020-11-15')
    .as_dataframe()
)

In [ ]:
df.head()

In [ ]:
for region in ['moscow', 'saint-petersburg', 'other']:

    df_filtered = df[
        (df['region']==region)
        & (df['creation_local_datetime']<='2020-11-08')
    ]

    print(region, round(df_filtered['fulfillment_name'].value_counts().sum() / df_filtered.shape[0], 3))

In [ ]:
(df[
    (df['region']=='moscow')
    & (df['creation_local_datetime']<='2020-11-08')
]['marketplace_model_type'].value_counts() / df[
    (df['region']=='moscow')
    & (df['creation_local_datetime']<='2020-11-08')
]['marketplace_model_type'].value_counts().sum()).round(2)

In [ ]:
(df[
    (df['region']=='saint-petersburg')
    & (df['creation_local_datetime']<='2020-11-08')
]['marketplace_model_type'].value_counts() / df[
    (df['region']=='saint-petersburg')
    & (df['creation_local_datetime']<='2020-11-08')
]['marketplace_model_type'].value_counts().sum()).round(2)

In [ ]:
(df[
    (df['region']=='other')
    & (df['creation_local_datetime']<='2020-11-08')
]['marketplace_model_type'].value_counts() / df[
    (df['region']=='other')
    & (df['creation_local_datetime']<='2020-11-08')
]['marketplace_model_type'].value_counts().sum()).round(2)

In [ ]:
df['fulfillment_id'] = df['fulfillment_id'].fillna(999)

In [ ]:
df_filtered = df[(df['creation_local_datetime']<='2020-11-08')]
# df_filtered = df[
#     (df['creation_local_datetime']<='2020-11-15')
#     & (df['creation_local_datetime']>='2020-11-09')
# ]

In [ ]:
df_filtered.groupby(
    ['region', 'marketplace_model_type', 'fulfillment_id',]
).count()[['creation_datetime']]

In [ ]:
df_filtered.groupby(
    ['region', 'marketplace_model_type', 'fulfillment_id',]
).count()[['creation_datetime']]

In [ ]:
df['middle_mile_delivery_service_name'].value_counts()